In [26]:
import pandas as pd
import re

In [4]:
with open('Finlab.txt',encoding="utf-8") as f:
    lines = f.readlines()

In [9]:
lines[:10]

['回測基準\thttps://ai.finlab.tw/database/#benchmark_return\t"資料名稱 使用方法 型態\n',
 '發行量加權股價報酬指數 data.get(\'benchmark_return:發行量加權股價報酬指數\') float"\n',
 '上櫃減資\thttps://ai.finlab.tw/database/#capital_reduction_otc\t"資料名稱 使用方法 型態\n',
 "恢復買賣日期 data.get('capital_reduction_otc:恢復買賣日期') str\n",
 "減資原因 data.get('capital_reduction_otc:減資原因') str\n",
 "開始交易基準價 data.get('capital_reduction_otc:開始交易基準價') float\n",
 "最後交易之收盤價格 data.get('capital_reduction_otc:最後交易之收盤價格') float\n",
 "減資恢復買賣開始日參考價格 data.get('capital_reduction_otc:減資恢復買賣開始日參考價格') float\n",
 "漲停價格 data.get('capital_reduction_otc:漲停價格') float\n",
 "跌停價格 data.get('capital_reduction_otc:跌停價格') float\n"]

In [18]:
output = {'data_name':[i.split(' ')[0] for i in lines], 'api':[i.split(' ')[1] for i in lines], 'd_type':[i.split(' ')[2].strip('\n').strip('"') for i in lines]}

In [20]:
output = pd.DataFrame(output)

In [29]:
output.head(5)

,data_name,api,d_type
0,回測基準\thttps://ai.finlab.tw/database/#benchmark...,使用方法,型態
1,發行量加權股價報酬指數,data.get('benchmark_return:發行量加權股價報酬指數'),float
2,上櫃減資\thttps://ai.finlab.tw/database/#capital_r...,使用方法,型態
3,恢復買賣日期,data.get('capital_reduction_otc:恢復買賣日期'),str
4,減資原因,data.get('capital_reduction_otc:減資原因'),str


In [25]:
category = output[output['api']=='使用方法']

In [28]:
category.head(5)

,data_name,api,d_type
0,回測基準\thttps://ai.finlab.tw/database/#benchmark...,使用方法,型態
2,上櫃減資\thttps://ai.finlab.tw/database/#capital_r...,使用方法,型態
12,上市減資\thttps://ai.finlab.tw/database/#capital_r...,使用方法,型態
22,企業基本資訊\thttps://ai.finlab.tw/database/#company...,使用方法,型態
64,企業主要經營業務\thttps://ai.finlab.tw/database/#compa...,使用方法,型態


In [40]:
def extract_category(input_str):
    '''
    Correct the wrong data names that are actually category names.
    
    Args: 
    input_str: wrong data_names
    
    Return:
    A list of categories
    '''
    end = re.search('\thttps', input_str).span()[0]
    category = input_str[:end]
    
    return category

In [50]:
category_obj = category['data_name'].apply(lambda x: extract_category(x))

In [57]:
def make_output_dataframe(cat, df):
    '''
    Make extra one column category to output dataframe and remove redundant rows.
    
    Args: 
    cat: category object containing the index and the information about the category |0 回測基準 |2 上櫃減資|...
    df: the dataframe which has been processed primarily.
    
    Return:
    the final desired dataframe containing name, api, dtype, and category
    '''
    df['category'] = '' # Initialize a new column
    for cat_index in cat.index:
        df['category'][cat_index] = cat[cat_index]
    for df_index in df.index:
        if df['category'][df_index]=='':
            df['category'][df_index] = df['category'][df_index-1]
    
    #Above process fill up the categories
    
    output_df = df.drop(cat.index, axis = 0).sort_index()
    
    return output_df

In [59]:
output_df = make_output_dataframe(category_obj, output)

In [60]:
output_df.to_csv('finlab_api.csv', encoding = 'utf-8-sig', format)